# 02 - Data from the Web

In [1]:
# Import libraries
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

You will need requests-futures which you can get by   
$ ``pip install requests-futures``

### Task 1.0 Retrieve the Data
Obtain the 200 top-ranking universities : name, rank, country and region, number of faculty members (international and total) and number of students (international and total)

In [2]:
# Do the request
request_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1507975779496'
r = requests.get(request_url)

In [3]:
univ_vals = r.json()['data']
univ_vals[0]

{'cc': 'US',
 'core_id': '410',
 'country': 'United States',
 'guide': '<a href="/where-to-study/north-america/united-states/guide" class="guide-link" target="_blank">United States</a>',
 'logo': '<img src="https://www.topuniversities.com/sites/default/files/massachusetts-institute-of-technology-mit_410_small_0.jpg" alt="Massachusetts Institute of Technology (MIT)  Logo">',
 'nid': '294850',
 'rank_display': '1',
 'region': 'North America',
 'score': '100',
 'stars': '6',
 'title': 'Massachusetts Institute of Technology (MIT)',
 'url': '/universities/massachusetts-institute-technology-mit'}

In [4]:
univ_df = pd.DataFrame(univ_vals)
univ_df.head()

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,US,253,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,US,94,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,GB,95,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge


keep only : 
1. name, rank, country and region
2. number of faculty members (international and total) and number of students (international and total)

second part must be retrieved from different request

In [5]:
univ_dr_df = univ_df.drop('guide', 1).drop('logo', 1).drop('stars', 1).drop('url', 1) \
.drop('score', 1).drop('cc', 1).drop('core_id', 1).drop('nid', 1)
univ_dr_df.head()

,country,rank_display,region,title
0,United States,1,North America,Massachusetts Institute of Technology (MIT)
1,United States,2,North America,Stanford University
2,United States,3,North America,Harvard University
3,United States,4,North America,California Institute of Technology (Caltech)
4,United Kingdom,5,Europe,University of Cambridge


In [6]:
url_caltech = 'https://www.topuniversities.com/universities/california-institute-technology-caltech#wurs'
r_caltech = requests.get(url_caltech)
soup_caltech = BeautifulSoup(r_caltech.text, 'html.parser')
soup_caltech.prettify()[0:600]

'<!DOCTYPE html>\n<html dir="ltr" version="XHTML+RDFa 1.0" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/ns/book#" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#" xmlns:profile="http://ogp.me/ns/profile#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:schema="http://schema.org/" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xm'

From the soup, with a quick ctrl+f on the number displayed on the corresponding page, I was able to identify the elements which contained the value I was interested in:
1. Number of international students ``<div class="int-students-main wrapper col-md-4"> \ <div class="number">``
    
2. Number of students ``<div class="students-main wrapper col-md-4"> \ <div class="number">``

3. Number of faculty staff in total ``<div class="faculty-main wrapper col-md-4"> \ <div class="number">``

4. Number of faculty staff international ``<div class="inter faculty"> \ <div class="number">``

In [7]:
caltech_inter_st = soup_caltech.find('div', class_='int-students-main wrapper col-md-4').find('div', class_='number').text
caltech_inter_st[1:]

'647 '

In [8]:
caltech_st = soup_caltech.find('div', class_ = 'students-main wrapper col-md-4').find('div', class_='number').text
caltech_st[1:]

'2,255 '

Now we can test with MIT to see whether the parsing works the same (which is probably the case). If it works, then we will iterate over the links in the dataframe to parse the values of interest for each university

In [9]:
url_mit = 'https://www.topuniversities.com/universities/massachusetts-institute-technology-mit#wurs'
r_mit = requests.get(url_mit)
soup_mit = BeautifulSoup(r_mit.text, 'html.parser')
soup_mit.find('div', class_='int-students-main wrapper col-md-4').find('div', class_='number').text[1:]


'3,717 '

In [385]:
#tries to retrieve the value contained in the child <block_p class = 'class_p'>, hence contained in <block_c class='class_c'
#returns NaN if one of both block is not found
def retrieve(soup, block_p, class_p, block_c, class_c):
    parent = soup.find(block_p, class_p)
    if parent is None:
        return np.NaN
    else:
        child = parent.find(block_c, class_c)
        if child is None:
            return np.NaN
        else:
            return int(child.text[1:].replace(',',''))
    

In [386]:
retrieve(soup_mit, 'div', 'int-students-main wrapper col-md-4', 'div', 'number')

3717

In [387]:
retrieve(soup_mit, 'div', 'int-st-main wrapper col-md-4', 'div', 'number')

nan

In [388]:
retrieve(soup_mit, 'div', 'int-students-main wrapper col-md-4', 'div', 'nber')

nan

In [389]:
def retrieve_div_number(soup, class_p):
    return retrieve(soup, 'div', class_p, 'div', 'number')

In [390]:
retrieve_div_number(soup_mit, 'int-students-main wrapper col-md-4')

3717

In [391]:
u_urls = np.array(univ_df[['url']].values)
univ_urls = []
main_url = 'https://www.topuniversities.com'
for url_ in u_urls:
    url = url_[0]
    univ_urls.append(main_url + url + '#wurs')
univ_urls[:5]

['https://www.topuniversities.com/universities/massachusetts-institute-technology-mit#wurs',
 'https://www.topuniversities.com/universities/stanford-university#wurs',
 'https://www.topuniversities.com/universities/harvard-university#wurs',
 'https://www.topuniversities.com/universities/california-institute-technology-caltech#wurs',
 'https://www.topuniversities.com/universities/university-cambridge#wurs']

In [392]:
len(univ_urls)

959

Since doing the 959 requests in one go didn't work, I will manually split the requests in chunks as big as possible to minimize the merge step

In [393]:
from requests_futures.sessions import FuturesSession

session = FuturesSession()

In [394]:
def get_missing_data(urls):
    international_students = []
    students = []
    international_faculty = []
    faculty = []

    for url in urls:
    
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
    
        inter_st = retrieve_div_number(soup, 'int-students-main wrapper col-md-4')
        st = retrieve_div_number(soup, 'students-main wrapper col-md-4')
        inter_fac = retrieve_div_number(soup, 'faculty-main wrapper col-md-4')
        fac = retrieve_div_number(soup, 'inter faculty')
    
        international_students.append(inter_st)
        students.append(st)
        international_faculty.append(inter_fac)
        faculty.append(fac)

    return international_students, students, international_faculty, faculty

In [395]:
inter_s, s, inter_f, f = get_missing_data(univ_urls[:5])

In [396]:
inter_s

[3717, 3611, 5266, 647, 6699]

Since this is super slow, we will use the package future-requests that will make non-blocking requests. While waiting, I will treat only a small part of the data. When requests complete, I store the results in variables and its all good. This is the reason for the small block before the function

In [626]:
international_st = []
students = []
international_fac = []
fac = []
count = 0

In [627]:
def callback(future):
    r = future.result()
    soup = BeautifulSoup(r.text, 'html.parser')
    
    global international_st
    global students
    global international_fac
    global fac
    global num_request
    global count
    
    count = count + 1
    if count > num_request:
        international_st = []
        students = []
        international_fac = []
        fac = []
        count = 0
    
    inter_st = retrieve_div_number(soup, 'int-students-main wrapper col-md-4')
    st = retrieve_div_number(soup, 'students-main wrapper col-md-4')
    inter_fac = retrieve_div_number(soup, 'faculty-main wrapper col-md-4')
    f = retrieve_div_number(soup, 'inter faculty')        
    
    international_st.append(inter_st)
    students.append(st)
    international_fac.append(inter_fac)
    fac.append(f)

Now we can do our 959 requests in background. For it to work, we must recompile the initilisation of arrays as well as the callback function

The following block will play a small sound when the requests are done. For it to work you need to install python-vlc   
$ pip install python-vlc

In [628]:
import vlc
def play_sound(future):
    folder_path = 'fancy_sounds/'
    music_path = 'zelda_small_item.wav'
    p = vlc.MediaPlayer(folder_path + music_path)
    p.play()
    return 0

In [630]:
url_slice = univ_urls[:10]
num_request = len(url_slice)

for i_url, url in enumerate(url_slice):
    # On fait notre requête GET
    future = session.get(url)
    # On rajoute le callback à appeler quand
    # le résultat de la requête arrive.
    future.add_done_callback(callback)
    if i_url == (len(url_slice) - 1):
        future.add_done_callback(play_sound)
        


Si vous lancez de nouvelles requête avant que les précédentes soient terminées, vous avez de forte chances que la data soit corrompue. Dans ce cas attendez la fin des requêtes en cours, puis relancez la boucle ci-dessus

In [612]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    global fac
    while(len(fac) <= num_request):
        progress.value = len(fac)/num_request
thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

In [576]:
len(fac)

1

If you compile the line above, if you have correctly init the requests, you should see the length of the array increase. This is proof that the data is indeed loading

Now that you finished your coffee and were alerted by the fancy sounds that you could pursue the compilation, we can finally use our well-deserved data

In [631]:
missing_df = pd.DataFrame(data= [international_st, students, international_fac, fac],
                         index = ['inter_st', 'students', 'inter_fac', 'fac']).transpose()
missing_df.head()

,inter_st,students,inter_fac,fac
0,3611,15878,4285,2042
1,3717,11067,2982,1679
2,5266,22429,4350,1311
3,647,2255,953,350
4,6699,18770,5490,2278


In [632]:
univ_tot_df = pd.concat([univ_dr_df, missing_df], axis = 1)
univ_rk_df = univ_tot_df.set_index(['rank_display'])
univ_rk_df.head()

,country,region,title,inter_st,students,inter_fac,fac
rank_display,,,,,,,
1,United States,North America,Massachusetts Institute of Technology (MIT),3611.0,15878.0,4285.0,2042.0
2,United States,North America,Stanford University,3717.0,11067.0,2982.0,1679.0
3,United States,North America,Harvard University,5266.0,22429.0,4350.0,1311.0
4,United States,North America,California Institute of Technology (Caltech),647.0,2255.0,953.0,350.0
5,United Kingdom,Europe,University of Cambridge,6699.0,18770.0,5490.0,2278.0


### Task 1.1 
Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

(a) ratio between faculty members and students

#### Warning
Le dataframe contient d

In [637]:
univ_ratio = univ_tot_df.copy().dropna()
univ_ratio['ratio'] = univ_ratio['students'] / univ_ratio['fac']
univ_ratio_sorted = univ_ratio.sort_values(['ratio'], ascending = 1)
univ_ratio_sorted.head()

,country,rank_display,region,title,inter_st,students,inter_fac,fac,ratio
3,United States,4,North America,California Institute of Technology (Caltech),647.0,2255.0,953.0,350.0,6.442857
1,United States,2,North America,Stanford University,3717.0,11067.0,2982.0,1679.0,6.591423
5,United Kingdom,6,Europe,University of Oxford,7353.0,19720.0,6750.0,2964.0,6.653171
7,United Kingdom,8,Europe,Imperial College London,8746.0,16090.0,3930.0,2071.0,7.769194
0,United States,1,North America,Massachusetts Institute of Technology (MIT),3611.0,15878.0,4285.0,2042.0,7.775710
